### Stage 2: App behaviour is normally distributed

    1) The activity vectors are sampled from a Gaussian distribution
    2) The clusters with normal distributions represent samples generated from a hidden Markov process
    3) To test this, we must first observe whether the data can be split into clusters with Gaussian distributions at a significance level of 0.001
    4) This test will be done via the G-Means algorithm [1]
    
[1] Hamerly, G., & Elkan, C. (n.d.). Learning the k in k -means.


In [1]:
import numpy as np
import itertools

from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn import mixture

from utils import *
import gmeans
import gmpp
import pandas as pd

import plotly.plotly as py
import plotly.graph_objs as go

from sklearn.cluster import KMeans

In [2]:
time_percentage = 0.8
explained_variance = 0.8
df = pd.read_csv("data/rescuetime_data-ac-min.csv")
data_pd = Clean_DF(df)
data_pd.clean_data(time_percentage=time_percentage)
data_pd.clean_df = data_pd.clean_df.reset_index()
data_pd.get_pca(explained_variance=explained_variance)
data_pd.get_day_time()

## G-Means

#### Algorithm: 

    1) Start from k=1      
    2) Test whether data in each cluster is a Gaussian via the Anderson-Darling test at alpha = 0.0001       
    3) if data is Gaussian, keep center      
    4) else, split cluster into two sub clusters and repeat from 2      


#### Gaussian test:      
    
    1) Run k-means on sub-data X with k=2      
    2) Compute v = c1 - c2 where c1 & c2 are the two new cluster centres found by k-means on X 
    3) Project X onto v Xv = dot(X,v)/dot(v,v)      
    4) Scale Xv to mean 0 and unit variance      
    5) apply Anderson-Darling test to this vector      

In [3]:
# from scipy.stats import f
# print(np.sqrt(f.ppf(0.99, 2, 10000, loc=0, scale=1)))

In [4]:
print(cm.cluster_centers_)

NameError: name 'cm' is not defined

In [5]:
A = np.random.normal(loc=(5,5), scale=(2,2), size=(1000,2))
B = np.random.normal(loc=(1,22), scale=(2,2), size=(1000,2))
C = np.random.normal(loc=(30,10), scale=(2,2), size=(2000,2))
D = np.random.normal(loc=(18,8), scale=(3,3), size=(2000,2))
E = np.random.normal(loc=(50,50), scale=(3,3), size=(1000,2))

X = np.concatenate((A,B,C,D,E))

In [7]:
cm = gmpp.GMeans(verbose=0, trim_points=True, recalculate_points=True, significance_level=0.99, critical_value = 1.865)
cm.fit(X)
print(len(cm.cluster_centers_))
print(cm.cluster_centers_)

/Users/asturkmani/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning:

Explicit initial center position passed: performing only one init in k-means instead of n_init=10



6
[[ 49.98156021  50.20991596]
 [  1.07308296  22.0758027 ]
 [ 19.02559005  10.15671739]
 [  4.99334129   5.01275083]
 [ 17.00990126   5.87141711]
 [ 29.94056788   9.98738941]]


In [10]:
gm = gmeans.GMeans(verbose=0, trim_points=False)
gm.fit(X)
print(len(gm.cluster_centers_))
gaussian_clusters_gm = gm.cluster_centers_

2


/Users/asturkmani/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning:

Explicit initial center position passed: performing only one init in k-means instead of n_init=10



In [ ]:
print(gm.cluster_centers_)
kmeans = KMeans(n_clusters=len(gaussian_clusters), init=gaussian_clusters)
kmeans.fit(D)

c = gm.labels_
trace = go.Scatter(x=X[:,0], y=X[:,1], mode='markers', marker=dict(
        size='10',
        color = c, #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    ))
data= [trace]
py.iplot(data, filename='K-Means inerta')

In [ ]:
kmeans = KMeans(n_clusters=len(gaussian_clusters), init=gaussian_clusters)
kmeans.fit(D)

c = cm.labels_
trace = go.Scatter(x=X[:,0], y=X[:,1], mode='markers', marker=dict(
        size='10',
        color = c, #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    ))
data= [trace]
py.iplot(data, filename='K-Means inerta')

In [ ]:
cm = Cantankerousmeans.GMeans(verbose=0, trim_points=True, recalculate_points=True, significance_level=0.99, critical_value = 1.865)
cm.fit(data_pd.pca_data)
print(len(cm.cluster_centers_))
print(cm.cluster_centers_)

In [ ]:
c = cm.labels_
x = data_pd.pca_data[:,0]
y = data_pd.pca_data[:,1]
z = data_pd.pca_data[:,2]
t = data_pd.clean_df['Activity']
t = data_pd.clean_df['Activity'].tolist()
t = ['-'.join(x) for x in t]
for i in range(0,len(t)):
    t[i] = str(cm.labels_[i]) + '---' + t[i]
    
trace1 = go.Scatter3d(x=x,y=y,z=z,text=t,mode='markers',marker=dict(size=12,color=c, colorscale='Viridis',opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='PCA-3 Visualization with G-Means')

In [ ]:
from sklearn.manifold import TSNE
tsne_pca = TSNE(n_components=3, verbose=0, perplexity=30, n_iter=5000)
tsne_results_pca = tsne_pca.fit_transform(data_pd.pca_data)

In [ ]:
c = gm.labels_
x = tsne_results_pca[:,0]
y = tsne_results_pca[:,1]
z = tsne_results_pca[:,2]
# t = data_pd.clean_df['Activity']
# t = data_pd.clean_df['Activity'].tolist()
# t = ['-'.join(x) for x in t]

trace1 = go.Scatter3d(x=x,y=y,z=z,text=t,mode='markers',marker=dict(size=12,color=c, colorscale='RdYlGn',opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='t-SNE PCA 90% variance Visualization')

In [ ]:
lowest_bic = np.infty
bic = []
n_components_range = range(1, 20)
cv_types = ['spherical', 'tied', 'diag', 'full']
X = data_pd.pca_data
for cv_type in cv_types:
    for n_components in n_components_range:
        # Fit a Gaussian mixture with EM
        gmm = mixture.GaussianMixture(n_components=n_components,
                                      covariance_type=cv_type)
        gmm.fit(X)
        bic.append(gmm.bic(X))
        if (bic[-1] < lowest_bic):
            lowest_bic = bic[-1]
            best_gmm = gmm

bic = np.array(bic)
color_iter = itertools.cycle(['navy', 'turquoise', 'cornflowerblue',
                              'darkorange'])
clf = best_gmm
bars = []
lowest_bic = max(bic)

In [ ]:
print(bic)
# Plot the BIC scores
spl = plt.subplot(2, 1, 1)
for i, (cv_type, color) in enumerate(zip(cv_types, color_iter)):
    xpos = np.array(n_components_range) + .2 * (i - 2)
    bars.append(plt.bar(xpos, bic[i * len(n_components_range):
                                  (i + 1) * len(n_components_range)],
                        width=.2, color=color))
plt.xticks(n_components_range)
plt.ylim([bic.min() * 1.01 - .01 * bic.max(), bic.max()])
plt.title('BIC score per model')
xpos = np.mod(bic.argmin(), len(n_components_range)) + .65 +\
    .2 * np.floor(bic.argmin() / len(n_components_range))
plt.text(xpos, bic.min() * 0.97 + .03 * bic.max(), '*', fontsize=14)
spl.set_xlabel('Number of components')
spl.legend([b[0] for b in bars], cv_types)

plt.show()